In [1]:
"""
Avenue 1 — Classical Econometrics (Option B month-end snap)
- Cleans your exact CSV schemas
- Weekly returns (Fri)
- Monthly rebal carry (rank by 1M deposit rates EUR/JPY/GBP/CAD)
- Composite skew ("fear") from 25Δ RR (vs USD), expanding z
- Predict next-4-week carry with Newey–West SEs
"""

# ====== Imports ======
import pandas as pd
import numpy as np
from statsmodels.api import OLS, add_constant
from statsmodels.stats.sandwich_covariance import cov_hac


# ====== USER INPUTS ======
PATH_SPOT    = "Data/Daily_Spot_Prices_G10_FX_Pairs_Daily_2000_2025.csv"
PATH_IR_1M   = "Data/AT-21_FX_Forward_Points_1M_and_3M_Major_Pairs_Daily_2005_2025.csv"  # using 1M "Curncy" levels
PATH_RR      = "Data/AT-32_Risk_Reversal_Daily_2005_2025.csv"
PATH_VIX     = "Data/AT-33_Stress_Indicies_Daily_1999_2025.csv"

WEEKLY_FREQ         = "W-FRI"   # align to Friday
HORIZON_WEEKS       = 4         # next-4-week target (overlapping)
MIN_EXPANDING_WEEKS = 52        # for z-scores
WARNING_QUANTILE    = 0.10      # bottom decile as "warning"

# ====== 1) LOAD & CLEAN ======
spot_df = pd.read_csv(PATH_SPOT)
ir_df   = pd.read_csv(PATH_IR_1M)
rr_df   = pd.read_csv(PATH_RR)
vix_df = pd.read_csv(PATH_VIX)

In [2]:
vix_df.columns

Index(['Date', 'BASPTDSP Index  (L1)', 'VIX Index  (R1)'], dtype='object')

In [3]:
vix = vix_df.copy()
vix["Date"] = pd.to_datetime(vix["Date"])
vix = vix.set_index("Date").sort_index()

# Resample to Friday weekly frequency to match your carry data
vix_w = vix.resample("W-FRI").last()

# Rename the column to something cleaner
vix_w.rename(columns={"VIX Index  (R1)": "VIX"}, inplace=True)

In [4]:
def extract_series(df, date_col, value_col, new_name):
    out = df[[date_col, value_col]].copy()
    out.columns = ["Date", new_name]
    out["Date"] = pd.to_datetime(out["Date"])
    return out.set_index("Date").sort_index()


In [5]:
# ---- Spot (USD pairs) ----
ser_EURUSD = extract_series(spot_df, "EURUSD - Date",  "EURUSD -Last Price", "EURUSD")
ser_USDJPY = extract_series(spot_df, "USDJPY - Date",  "USDJPY - Last Price", "USDJPY")
ser_GBPUSD = extract_series(spot_df, "GBPUSD - Date",  "GBPUSD - Last Price", "GBPUSD")
ser_USDCAD = extract_series(spot_df, "USDCAD - Date",  "USDCAD - Last Price", "USDCAD")
ser_AUDUSD = extract_series(spot_df, "AUDUSD - Date",  "AUDUSD - Last Price", "AUDUSD")
ser_USDCHF = extract_series(spot_df, "USDCHF - Date",  "USDCHF - Last Price", "USDCHF")

spot = pd.concat(
    [ser_EURUSD, ser_USDJPY, ser_GBPUSD, ser_USDCAD, ser_AUDUSD, ser_USDCHF],
    axis=1
)

In [6]:
# ---- 1M "Curncy" (deposit) rates ----
ser_EUR1M = extract_series(ir_df, "EUR1M Curncy - Date", "EUR1M Curncy - Last Price", "EUR")
ser_JPY1M = extract_series(ir_df, "JPY1M Curncy - Date", "JPY1M Curncy - Last Price", "JPY")
ser_GBP1M = extract_series(ir_df, "GBP1M Curncy - Date", "GBP1M Curncy - Last Price", "GBP")
# Note: double-space before 'Date' in your source column
ser_CAD1M = extract_series(ir_df, "CAD1M Curncy -  Date", "CAD1M Curncy - Last Price", "CAD")

rates = pd.concat([ser_EUR1M, ser_JPY1M, ser_GBP1M, ser_CAD1M], axis=1)

# ---- Risk reversals (25Δ) ----
rr = rr_df.copy()
rr["Date"] = pd.to_datetime(rr["Date"])
rr = rr.set_index("Date").sort_index()

# Normalize so positive means "calls on Ccy vs USD richer than puts"
rr_inputs = pd.DataFrame(index=rr.index)
if "EURUSD" in rr.columns: rr_inputs["EUR"] = rr["EURUSD"]
if "GBPUSD" in rr.columns: rr_inputs["GBP"] = rr["GBPUSD"]
if "AUDUSD" in rr.columns: rr_inputs["AUD"] = rr["AUDUSD"]
if "USDJPY" in rr.columns: rr_inputs["JPY"] = -rr["USDJPY"]
if "USDCAD" in rr.columns: rr_inputs["CAD"] = -rr["USDCAD"]
# (Add CHF/NZD/etc when available)

In [7]:
# ====== 2) RESAMPLE TO WEEKLY (Friday) ======
spot_w  = spot.resample(WEEKLY_FREQ).last()
rates_w = rates.resample(WEEKLY_FREQ).last()
rr_w    = rr_inputs.resample(WEEKLY_FREQ).last()

# ====== 3) PER-CCY WEEKLY LOG RETURNS (vs USD, long-ccy convention) ======
logret = pd.DataFrame(index=spot_w.index)
if "EURUSD" in spot_w: logret["EUR"] = np.log(spot_w["EURUSD"]).diff()
if "GBPUSD" in spot_w: logret["GBP"] = np.log(spot_w["GBPUSD"]).diff()
if "AUDUSD" in spot_w: logret["AUD"] = np.log(spot_w["AUDUSD"]).diff()
if "USDCAD" in spot_w: logret["CAD"] = -np.log(spot_w["USDCAD"]).diff()
if "USDJPY" in spot_w: logret["JPY"] = -np.log(spot_w["USDJPY"]).diff()
# (CHF later: long CHF = -Δln(USDCHF))

In [8]:
# ====== 4) MONTHLY REBALANCED CARRY WEIGHTS (Option B) ======
rankable = ["EUR","JPY","GBP","CAD"]
rates_w_rank = rates_w[rankable].copy()

# Snap to true month-ends, then forward-fill from the last weekly obs
month_ends = rates_w_rank.resample("ME").last().index  # 'ME' avoids FutureWarning
rates_m = rates_w_rank.reindex(month_ends, method="pad")  # forward-fill to exact ME stamps

def make_weights_from_rates(r_row, n_long=2, n_short=2):
    # Rank by level (higher rate = long), equal-weight long/short, dollar-neutral
    s = r_row.dropna().sort_values(ascending=False)
    long = s.index[:n_long]; short = s.index[-n_short:]
    w = pd.Series(0.0, index=r_row.index)
    if len(long)  > 0: w[long]  = +1.0 / len(long)
    if len(short) > 0: w[short] = -1.0 / len(short)
    return w

w_m = rates_m.apply(make_weights_from_rates, axis=1)

# Hold monthly weights until next month-end; project back to weekly grid
w_w = w_m.reindex(rates_w_rank.index).ffill().fillna(0.0)

# Carry portfolio weekly log return
carry_ret_w = (w_w[rankable] * logret[rankable]).sum(axis=1)

In [9]:
# ====== 5) COMPOSITE SKEW (“FEAR”) ======
def expanding_zscore(x, min_periods=MIN_EXPANDING_WEEKS):
    mu = x.expanding(min_periods=min_periods).mean()
    sd = x.expanding(min_periods=min_periods).std()
    return (x - mu) / sd

rr_z = rr_w.apply(expanding_zscore)

long_mask  = (w_w[rankable] > 0).astype(float)
short_mask = (w_w[rankable] < 0).astype(float)

def bucket_avg(values, mask):
    numer = (values * mask).sum(axis=1)
    denom = mask.sum(axis=1).replace(0, np.nan)
    return numer / denom

comp_skew = bucket_avg(rr_z[rankable], long_mask) - bucket_avg(rr_z[rankable], short_mask)
dfear = comp_skew.diff()


In [10]:
# ====== 6) NEXT-4-WEEK FORWARD TARGET (overlapping) ======
target_4w = carry_ret_w.shift(-1).rolling(HORIZON_WEEKS).sum()

# Align predictors/target
X = pd.DataFrame({"dfear": dfear}).dropna()
y = target_4w.reindex(X.index).dropna()
X = X.reindex(y.index)


In [11]:
# ====== 7) PREDICTIVE REGRESSION + NEWEY–WEST SEs ======
Xc = add_constant(X)
model = OLS(y, Xc, missing="drop").fit()
hac_lags = max(HORIZON_WEEKS - 1, 0)
V = cov_hac(model, nlags=hac_lags)
se = np.sqrt(np.diag(V))

print("=== Predictive Regression: Next 4-week carry (log) ===")
for name, b, s in zip(model.params.index, model.params.values, se):
    t = b / s if s > 0 else np.nan
    print(f"{name:>8}: {b: .6f}   (SE {s: .6f})   t={t: .2f}")
print(f"R^2: {model.rsquared:.4f}   N={int(model.nobs)}")

=== Predictive Regression: Next 4-week carry (log) ===
   const:  0.000685   (SE  0.001012)   t= 0.68
   dfear:  0.002816   (SE  0.002366)   t= 1.19
R^2: 0.0075   N=1041


In [12]:
# ====== 8) EVENT STUDY AROUND “WARNING” (bottom decile of dfear) ======
q = dfear.quantile(WARNING_QUANTILE)
events = dfear.index[dfear <= q]

ev_mat = []
for t0 in events:
    # window -4..+4 around event (inclusive)
    loc = carry_ret_w.index.get_loc(t0)
    if isinstance(loc, slice):   # guard (rare duplicate index)
        loc = loc.start
    start = max(0, loc - 4)
    end   = min(len(carry_ret_w) - 1, loc + 4)
    seg   = carry_ret_w.iloc[start:end+1].cumsum()
    seg   = seg - seg.iloc[0]   # rebase to 0 at event-4
    rel_idx = np.arange(start - loc, end - loc + 1)
    seg.index = rel_idx
    ev_mat.append(seg)

if ev_mat:
    common_idx = sorted(
        set(ev_mat[0].index).intersection(*[set(s.index) for s in ev_mat[1:]])
    ) if len(ev_mat) > 1 else list(ev_mat[0].index)
    ev_avg = pd.concat([s.reindex(common_idx) for s in ev_mat], axis=1).mean(axis=1)
    print("\nEvent study (avg cum log return around warnings):")
    print(ev_avg)
else:
    print("\nEvent study: no events found (adjust WARNING_QUANTILE or coverage).")


Event study (avg cum log return around warnings):
-4    0.000000
-3    0.000267
-2    0.001221
-1    0.002100
 0   -0.006518
 1   -0.007300
 2   -0.006651
 3   -0.006521
 4   -0.005295
dtype: float64


In [13]:
# --- Helpers ---
def forward_k_week_log_return(ret_series: pd.Series, k: int) -> pd.Series:
    """
    Overlapping forward k-week log return: sum of next k weekly log returns.
    y_t = r_{t+1} + ... + r_{t+k}
    """
    return ret_series.shift(-1).rolling(k).sum()

def newey_west_cov(res, k):
    """Version-robust HAC call (statsmodels differs on arg name)."""
    try:
        return cov_hac(res, maxlags=k)
    except TypeError:
        return cov_hac(res, nlags=k)

def standardize(df: pd.DataFrame) -> pd.DataFrame:
    """z-score columns (skip all-NaN or constant)."""
    out = df.copy()
    for c in out.columns:
        s = out[c]
        mu, sd = s.mean(), s.std()
        if np.isfinite(sd) and sd > 0:
            out[c] = (s - mu) / sd
    return out

def run_predictive_regression(y: pd.Series,
                              X: pd.DataFrame,
                              hac_lags: int):
    """
    Run OLS(y ~ const + X) and attach HAC SEs (Newey–West with hac_lags).
    Returns a tidy results DataFrame.
    """
    df = pd.concat([y.rename("y"), X], axis=1).dropna()
    if df.empty or df.shape[0] < 30:
        return pd.DataFrame()

    y_clean = df["y"]
    Xc = add_constant(df.drop(columns=["y"]))
    res = OLS(y_clean, Xc, missing="drop").fit()
    V = newey_west_cov(res, hac_lags)
    se = np.sqrt(np.diag(V))
    params = res.params
    tvals = params / se
    # p-values from asymptotic normal (conservative); or use res.t_test with HAC if you prefer
    from scipy.stats import norm
    pvals = 2 * (1 - norm.cdf(np.abs(tvals)))

    out = pd.DataFrame({
        "coef": params,
        "se_NW": se,
        "t_NW": tvals,
        "p_NW": pvals
    })
    out["R2"] = res.rsquared
    out["N"]  = int(res.nobs)
    return out

In [14]:
# =========================
# 1) Build controls (now including VIX)
# =========================
mom_4w = carry_ret_w.rolling(4).sum()               # momentum
vol_12w = carry_ret_w.rolling(12).std()             # realized vol
carry_strength = (w_w * rates_w.reindex(w_w.index)).sum(axis=1)  # rate differential
vix_aligned = vix_w.reindex(w_w.index)["VIX"]       # global risk proxy

controls_raw = pd.DataFrame({
    "mom_4w": mom_4w,
    "vol_12w": vol_12w,
    "carry_strength": carry_strength,
    "VIX": vix_aligned
})
controls = controls_raw

In [15]:
# =========================
# 2) Choose your SkewIndicator(s)
# =========================
# You can test either (or both):
#   - Level: comp_skew (more negative = “fearier”)
#   - Change: dfear (more negative = steepening toward puts)
skew_level = comp_skew.rename("Skew_level")
skew_change = dfear.rename("Skew_change")


In [16]:
# =========================
# 3) Run overlapping predictive regressions for multiple horizons
# =========================
k_list = [1, 4, 8, 12]  # weeks ahead (≈ 1w, 1m, 2m, 3m)
results = {}

for k in k_list:
    y_k = forward_k_week_log_return(carry_ret_w, k)

    # HAC lags: k-1 for overlap (Newey–West rule of thumb)
    hac_lags = max(k - 1, 0)

    # ----- Model A: Skew LEVEL only -----
    X_A = pd.concat([skew_level], axis=1)

    # ----- Model B: Skew CHANGE only -----
    X_B = pd.concat([skew_change], axis=1)

    # ----- Model C: Skew LEVEL + Controls -----
    X_C = pd.concat([skew_level, controls], axis=1)

    # ----- Model D: Skew CHANGE + Controls -----
    X_D = pd.concat([skew_change, controls], axis=1)

    res_A = run_predictive_regression(y_k, X_A, hac_lags)
    res_B = run_predictive_regression(y_k, X_B, hac_lags)
    res_C = run_predictive_regression(y_k, X_C, hac_lags)
    res_D = run_predictive_regression(y_k, X_D, hac_lags)

    results[k] = {
        "A_SkewLevel": res_A,
        "B_SkewChange": res_B,
        "C_SkewLevel_Controls": res_C,
        "D_SkewChange_Controls": res_D
    }


In [17]:
# =========================
# 4) Print tidy summaries & sign checks
# =========================
def print_block(title, df):
    if df is None or df.empty:
        print(f"{title}: (insufficient data)")
        return
    print("\n" + title)
    print(df.round(4))

for k, res_k in results.items():
    print("\n" + "="*70)
    print(f"Overlapping predictive regression: k = {k} weeks ahead")
    print("="*70)

    print_block("Model A — Skew LEVEL", res_k["A_SkewLevel"])
    if not res_k["A_SkewLevel"].empty and "Skew_level" in res_k["A_SkewLevel"].index:
        beta = res_k["A_SkewLevel"].loc["Skew_level","coef"]
        t    = res_k["A_SkewLevel"].loc["Skew_level","t_NW"]
        print(f"Expected sign: + (more negative skew ⇒ lower returns).  β={beta:.4f}, t={t:.2f}")

    print_block("Model B — Skew CHANGE", res_k["B_SkewChange"])
    if not res_k["B_SkewChange"].empty and "Skew_change" in res_k["B_SkewChange"].index:
        beta = res_k["B_SkewChange"].loc["Skew_change","coef"]
        t    = res_k["B_SkewChange"].loc["Skew_change","t_NW"]
        print(f"Expected sign: + (more negative Δskew ⇒ lower returns). β={beta:.4f}, t={t:.2f}")

    print_block("Model C — Skew LEVEL + Controls", res_k["C_SkewLevel_Controls"])
    if not res_k["C_SkewLevel_Controls"].empty and "Skew_level" in res_k["C_SkewLevel_Controls"].index:
        beta = res_k["C_SkewLevel_Controls"].loc["Skew_level","coef"]
        t    = res_k["C_SkewLevel_Controls"].loc["Skew_level","t_NW"]
        print(f"[Controls] Expected sign: +.  β={beta:.4f}, t={t:.2f}")

    print_block("Model D — Skew CHANGE + Controls", res_k["D_SkewChange_Controls"])
    if not res_k["D_SkewChange_Controls"].empty and "Skew_change" in res_k["D_SkewChange_Controls"].index:
        beta = res_k["D_SkewChange_Controls"].loc["Skew_change","coef"]
        t    = res_k["D_SkewChange_Controls"].loc["Skew_change","t_NW"]
        print(f"[Controls] Expected sign: +.  β={beta:.4f}, t={t:.2f}")


Overlapping predictive regression: k = 1 weeks ahead

Model A — Skew LEVEL
              coef   se_NW    t_NW    p_NW      R2     N
const       0.0002  0.0003  0.7615  0.4464  0.0047  1042
Skew_level  0.0007  0.0005  1.3990  0.1618  0.0047  1042
Expected sign: + (more negative skew ⇒ lower returns).  β=0.0007, t=1.40

Model B — Skew CHANGE
               coef   se_NW    t_NW    p_NW      R2     N
const        0.0002  0.0003  0.5511  0.5816  0.0038  1041
Skew_change  0.0010  0.0007  1.5068  0.1319  0.0038  1041
Expected sign: + (more negative Δskew ⇒ lower returns). β=0.0010, t=1.51

Model C — Skew LEVEL + Controls
                  coef   se_NW    t_NW    p_NW      R2     N
const           0.0010  0.0014  0.7037  0.4816  0.0099  1042
Skew_level      0.0007  0.0005  1.3803  0.1675  0.0099  1042
mom_4w         -0.0144  0.0221 -0.6521  0.5143  0.0099  1042
vol_12w        -0.0924  0.1090 -0.8480  0.3965  0.0099  1042
carry_strength  0.0000  0.0000  1.4534  0.1461  0.0099  1042
VIX        